# Notebook for Hot and Cold spot analysis with Copernicus Marine Data

**USER INPUT**

In [1]:
# define input dictionary for copernicus marine toolbox API, if other query desired just CHANGE dictionary input values

# dataset_id defines the product to be queried, by default set to sea surface temperature 
# variables defines a list of variables selected from the product (identifer = dataset_id), by default set to adjusted sea surface temperature
# max and min lat/lat define the bounding box of the query, by default set to the whole Agean sea 
# start/end_datetime define the time period covered by the dataset, by default set to winter 2023/24
input_dict = {
    "dataset_id": "SST_MED_SST_L3S_NRT_OBSERVATIONS_010_012_b", 
    "variables": ["adjusted_sea_surface_temperature"], 
    "minimum_longitude": 19.22659983450641, 
    "maximum_longitude": 28.439441984120553, 
    "minimum_latitude": 34.62160284496615, 
    "maximum_latitude": 40.9634662781889, 
    "start_datetime": "2023-12-01T00:00:00", 
    "end_datetime": "2024-02-28T00:00:00", 
}

In [2]:
# define units of your chosen data variable, since default variable is sea surface temperature => unit is set to degree Celsius 
unit = "°C"
# define abreviation of your data variable (coming from the server they usually have quite clumsy names), since defualt variable is sea surface temperature => abreviation is set to 'sst'
variable_abreviation = "sst"

DOI = "https://doi.org/10.48670/moi-00171"

spatial_resolution = "0.01° × 0.01°"

temporal_resolution = "Daily"

**Dependencies**

In [3]:
# necessary libraries to run this NOTEBOOK
# if not already installed, install via conda-forge channel using Ana(Mini)conda, Micromamba or pip or use requirements.yml in 
import copernicusmarine
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from esda.getisord import G_Local
from libpysal.weights import Queen
from pyproj import CRS
import contextily as ctx
import rioxarray as rio
import rasterio
import pathlib
import os
from rasterio.warp import Resampling

In [4]:
# retrieving realtive paths
NOTEBOOK_DIRECTORY = pathlib.Path().resolve()
OUTPUT_DIRECTORY = NOTEBOOK_DIRECTORY / "output" / "getis_plots"
BASIC_PLOTS = NOTEBOOK_DIRECTORY / "output" / "basic_plots"
DATA_DIRECTORY = NOTEBOOK_DIRECTORY / "output" / "data"

os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)
os.makedirs(DATA_DIRECTORY, exist_ok=True)
os.makedirs(BASIC_PLOTS, exist_ok=True)

## Functions

In [5]:
# function for fetching a copernicus marine dataset via 'lazy-loading' to NOTEBOOK
def get_cm_dataset(input_dict):
    dataset = copernicusmarine.open_dataset(
        dataset_id=input_dict["dataset_id"],
        variables=input_dict["variables"],
        minimum_longitude=input_dict["minimum_longitude"],
        maximum_longitude=input_dict["maximum_longitude"],
        minimum_latitude=input_dict["minimum_latitude"],
        maximum_latitude=input_dict["maximum_latitude"],
        start_datetime=input_dict["start_datetime"],
        end_datetime=input_dict["end_datetime"],
    )

    print("\nDataset fetched via API and loaded into RAM")

    return dataset

In [6]:
# function for pre-processing fetched dataset
def pre_processing(dataset, variable_abreviation):
    # renaming data variable
    dataset = dataset.rename({"adjusted_sea_surface_temperature": variable_abreviation})
    # converting Kelvin to Celsius
    dataset[variable_abreviation] = dataset[variable_abreviation]-273.15

    dataset = dataset.rio.write_crs(CRS.from_epsg(4326))

    dataset = dataset.rio.reproject(CRS.from_epsg(3857))

    print("\nDataset pre-processing finished")

    return dataset

In [7]:
# function for calculating basic stats on sst
def bstats(dataset_preprocessed, variable_abreviation):
    # computing mean
    dataset_mean = dataset_preprocessed[variable_abreviation].mean(dim="time", skipna=True)
    # computing median
    dataset_mean = dataset_preprocessed[variable_abreviation].median(dim="time", skipna=True)
    # computing standard deviation
    dataset_std = dataset_preprocessed[variable_abreviation].std(dim="time", skipna=True)

    print("Basic statistics calculation finished")

    return dataset_mean, dataset_mean, dataset_std

In [8]:
# function for computing GETIS ORD G* statistics
def getis_ord_g(dataset_median, variable_abreviation):
    # converting data array to pandas DataFrame 
    df = dataset_median.to_dataframe().reset_index()
    # droping NANs 
    df.dropna(subset=[variable_abreviation], inplace=True)
    # converting dataframe to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["x"], df["y"]))
    # computing Spatial Weights (Queen contiguity)
    w = Queen.from_dataframe(gdf, use_index=False)
    # applying Getis-Ord Gi*
    getisord = G_Local(gdf[variable_abreviation], w, star=True, transform="B")
    # computing Z-scores for hot/cold spots
    gdf["G"] = getisord.Zs  

    print("Calculation of GETIS-ORD G* statistic finished")

    return gdf

In [9]:
# function for calculating the 95th percentile of the dataset, filtering by values per pixel > 95th percentile, calculating the median over time
def percentile_5th(dataset, variable_abreviation):
    # computing the 95th percentile per pixel
    percentile_95th = dataset[variable_abreviation].quantile(0.95, dim="time")
    # masking values below 95th percentile
    top5 = dataset[variable_abreviation].where(dataset[variable_abreviation] >= percentile_95th)
    # calculating the median over time of values above  95th percentile
    top5_median = top5.median(dim="time", skipna=True)

    print("\nCalculation of 5% highest values finished")

    return top5_median

In [10]:
def visualization(dataset_mean, dataset_median, dataset_std, top5, input_dict, 
                  variable_abreviation, unit, DOI, temporal_resolution, spatial_resolution):
    """
    Generates and saves three visualizations (mean, median, std) with correct projection handling.

    Parameters:
        dataset_mean (xarray.DataArray): Mean values of the dataset.
        dataset_median (xarray.DataArray): Median values of the dataset.
        dataset_std (xarray.DataArray): Standard deviation values of the dataset.
        top5 (xarray.DataArray): 5% highest values.
        input_dict (dict): Dictionary containing metadata including start and end datetime.
        variable_abreviation (str): Variable abbreviation (e.g., SST for sea surface temperature).
        unit (str): Unit of measurement.
        DOI (str): Dataset DOI for attribution.
        temporal_resolution (str): Temporal resolution of dataset.
        spatial_resolution (str): Spatial resolution of dataset.
    """
    attribution = (f"Author: Moritz Mühlbauer 2025 in cooperation with Archipelagos\n"
                   f"Source: Generated using E.U. Copernicus Marine Service Information,\n"
                   f"DOI: {DOI}\n"
                   f"Dataset ID: {input_dict['dataset_id']}\n"
                   f"Spatial resolution: {spatial_resolution}\n"
                   f"Temporal resolution: {temporal_resolution}\n"
                   f"Projection: WEB MERCATOR (EPSG:3857)\n"
                   f"Basemap: Esri World Imagery"
                  )

    # Generate a timestamp for unique filenames
    start_date = input_dict["start_datetime"][:10]
    end_date = input_dict["end_datetime"][:10]

    # Get min and max values
    min_value = dataset_median.min().item()
    max_value = top5.max().item()

    # Define filenames (keys now match dataset names)
    filenames = {
        "Mean": f"mean_{variable_abreviation}_{start_date}-{end_date}.png",
        "Median": f"median_{variable_abreviation}_{start_date}-{end_date}.png",
        "Standard Deviation": f"std_{variable_abreviation}_{start_date}-{end_date}.png",
        "5% Highest Values": f"top5_{variable_abreviation}_{start_date}-{end_date}.png",
    }

    # Define dataset dictionary (keys match filenames)
    datasets = {
        "Mean": dataset_mean,
        "Median": dataset_median,
        "Standard Deviation": dataset_std,
        "5% Highest Values": top5,
    }
    
    # Define colormap dictionary
    colormaps = {
        "Mean": "RdBu_r",
        "Median": "RdBu_r",
        "Standard Deviation": "magma",
        "5% Highest Values": "RdBu_r",
    }
    
    # Define min/max limits for consistent color scaling
    vmin_vmax = {
        "Mean": (None, None),
        "Median": (min_value, max_value),
        "Standard Deviation": (None, None),
        "5% Highest Values": (min_value, max_value),
    }

    # Plot each dataset
    for key, data in datasets.items():
        fig, ax = plt.subplots(figsize=(12, 8))

        # Use xarray's plot function with a proper colorbar label
        data.plot(ax=ax, cmap=colormaps[key], vmin=vmin_vmax[key][0], vmax=vmin_vmax[key][1],
                  cbar_kwargs={'label': unit})  # <-- This sets the colorbar label

        ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, attribution="") 

        ax.set_title(f"{key} {variable_abreviation} ({start_date} - {end_date})\n")
        ax.set_xlabel("Easting (meters)")
        ax.set_ylabel("Northing (meters)")

        # Add attribution text
        ax.text(0.01, -0.18, attribution, fontsize=6, transform=ax.transAxes, 
                fontweight='normal', bbox={'facecolor': 'none', 'pad': 0.01, 'edgecolor': 'none'})

        # Save the figure
        plt.savefig(os.path.join(BASIC_PLOTS, filenames[key]), dpi=600, bbox_inches='tight')
        plt.close(fig)

    print(f"Plots saved in {BASIC_PLOTS}:")
    for file in filenames.values():
        print(f" - {file}")

In [11]:
# function for plotting getis statistic over the entire dataset
def getis_viz(gdf, input_dict, variable_abreviation):
    gdf = gdf.set_crs(CRS.from_epsg(3857))
    
    # Define metadata for attribution
    attribution = (f"Author: Moritz Mühlbauer 2025 in cooperation with Archipelagos\n"
                   f"Source: Generated using E.U. Copernicus Marine Service Information,\n"
                   f"DOI: {DOI}\n"
                   f"Dataset ID: {input_dict['dataset_id']}\n"
                   f"Spatial resolution: {spatial_resolution}\n"
                   f"Temporal resolution: {temporal_resolution}\n"
                   f"Projection: WEB MERCATOR (EPSG:3857)\n"
                   f"Basemap: Esri World Imagery")
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Add attribution text
    ax.text(1.02, 0.01, attribution, fontsize=5.5, transform=ax.transAxes, 
            fontweight='normal', bbox={'facecolor': 'none', 'pad': 0.01, 'edgecolor': 'none'})
    
    # Ensure bins are strictly within 7 categories
    min_G = gdf["G"].min()
    max_G = gdf["G"].max()
    bins = [-float("inf"), -2.58, -1.96, -1.65, 1.65, 1.96, 2.58, float("inf")]  # Strict binning

    gdf["Hotspot_Category"] = pd.cut(
    gdf["G"],
    bins=bins,
    labels=[
        "Very Cold (< -2.58)",
        "Cold Spot (-2.58 to -1.96)",
        "Moderate Cold (-1.95 to -1.65)",
        "Not Significant (-1.64 to 1.65)",
        "Moderate Hot (1.66 to 1.96)",
        "Hot Spot (1.97 to 2.58)",
        "Very Hot (> 2.58)"
    ],
    include_lowest=True
    )


    
    # Generate the plot
    gdf.plot(
        ax=ax,
        column="Hotspot_Category",
        cmap="coolwarm",
        legend=True,
        categorical=True,  # Ensure it's treated as a category
        edgecolor="none",
        markersize=0.3
    )
    
    # Fix the legend manually
    leg = ax.get_legend()
    if leg:
        legend_texts = leg.get_texts()[:7]  # Ensure only 7 labels are used
        for text, new_label in zip(legend_texts, gdf["Hotspot_Category"].cat.categories):
            text.set_text(new_label)


    
    ax.get_legend().set_bbox_to_anchor((1, 1))
    ax.get_legend().set_title(f"Getis-Ord Gi* Z-scores\n")
    
    # Add basemap
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, attribution="", zoom=8) 
    
    # Extract dates from input dictionary
    start_date = input_dict["start_datetime"][:10]
    end_date = input_dict["end_datetime"][:10]
    
    # Define title and filename
    title = f"Hot and Cold Spot Analysis of Median {variable_abreviation} / entire dataset ({start_date} - {end_date})\n"
    filename = f"getis_entire_dataset_{variable_abreviation}_{start_date}-{end_date}.png"
    
    plt.title(title)
    
    # Save figure
    output_fp = os.path.join(OUTPUT_DIRECTORY, filename)
    plt.savefig(output_fp, dpi=600, bbox_inches='tight')
    print(f"Plot of GETIS ORD G* Statistics saved to: {output_fp}")
    plt.close()

In [12]:
# function for plotting getis statistics over the 5 % highest values
def getis_top5_viz(gdf, input_dict, variable_abreviation):
    gdf = gdf.set_crs(CRS.from_epsg(3857))
    
    # Define metadata for attribution
    attribution = (f"Author: Moritz Mühlbauer 2025 in cooperation with Archipelagos\n"
                   f"Source: Generated using E.U. Copernicus Marine Service Information,\n"
                   f"DOI: {DOI}\n"
                   f"Dataset ID: {input_dict['dataset_id']}\n"
                   f"Spatial resolution: {spatial_resolution}\n"
                   f"Temporal resolution: {temporal_resolution}\n"
                   f"Projection: WEB MERCATOR (EPSG:3857)\n"
                   f"Basemap: Esri World Imagery")
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Add attribution text
    ax.text(1.02, 0.01, attribution, fontsize=5.5, transform=ax.transAxes, 
            fontweight='normal', bbox={'facecolor': 'none', 'pad': 0.01, 'edgecolor': 'none'})
    
    # Ensure bins are strictly within 7 categories
    min_G = gdf["G"].min()
    max_G = gdf["G"].max()
    bins = [-float("inf"), -2.58, -1.96, -1.65, 1.65, 1.96, 2.58, float("inf")]  # Strict binning

    gdf["Hotspot_Category"] = pd.cut(
    gdf["G"],
    bins=bins,
    labels=[
        "Very Cold (< -2.58)",
        "Cold Spot (-2.58 to -1.96)",
        "Moderate Cold (-1.95 to -1.65)",
        "Not Significant (-1.64 to 1.65)",
        "Moderate Hot (1.66 to 1.96)",
        "Hot Spot (1.97 to 2.58)",
        "Very Hot (> 2.58)"
    ],
    include_lowest=True
    )


    
    # Generate the plot
    gdf.plot(
        ax=ax,
        column="Hotspot_Category",
        cmap="coolwarm",
        legend=True,
        categorical=True,  # Ensure it's treated as a category
        edgecolor="none",
        markersize=0.3
    )
    
    # Fix the legend manually
    leg = ax.get_legend()
    if leg:
        legend_texts = leg.get_texts()[:7]  # Ensure only 7 labels are used
        for text, new_label in zip(legend_texts, gdf["Hotspot_Category"].cat.categories):
            text.set_text(new_label)


    
    ax.get_legend().set_bbox_to_anchor((1, 1))
    ax.get_legend().set_title(f"Getis-Ord Gi* Z-scores\n")
    
    # Add basemap
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, attribution="", zoom=8) 
    
    # Extract dates from input dictionary
    start_date = input_dict["start_datetime"][:10]
    end_date = input_dict["end_datetime"][:10]
    
    # Define title and filename
    title = f"Hot and Cold Spot Analysis of Median {variable_abreviation} / 5% highest values ({start_date} - {end_date})\n"
    filename = f"getis_top5_{variable_abreviation}_{start_date}-{end_date}.png"
    
    plt.title(title)
    
    # Save figure
    output_fp = os.path.join(OUTPUT_DIRECTORY, filename)
    plt.savefig(output_fp, dpi=600, bbox_inches='tight')
    print(f"Plot of GETIS ORD G* Statistics saved to: {output_fp}")
    plt.close()

In [13]:
# function to export getis statistics GeoDataFrames
def export_getis(getis_statistics, getis_top5, input_dict, variable_abreviation):

    # Generate a timestamp for unique filenames
    start_date = input_dict["start_datetime"][:10]
    end_date = input_dict["end_datetime"][:10]

    # assign CRS
    getis_statistics = getis_statistics.set_crs(CRS.from_epsg(3857))
    getis_top5 = getis_top5.set_crs(CRS.from_epsg(3857))

    # define filenames 
    filename_top5 = f"getis_top5_{variable_abreviation}_{start_date}-{end_date}.gpkg"
    filename_ed = f"getis_entire_dataset_{variable_abreviation}_{start_date}-{end_date}.gpkg"

    # create filepaths
    output_fp_top5 = os.path.join(DATA_DIRECTORY, filename_top5)
    output_fp_ed = os.path.join(DATA_DIRECTORY, filename_ed)
    
    # write to Geopackage
    getis_top5.to_file(output_fp_top5, driver="GPKG")
    getis_statistics.to_file(output_fp_ed, driver="GPKG")

    print(f"Datsets saved in {DATA_DIRECTORY}:")
    print(f"  - {filename_top5}")
    print(f"  - {filename_ed}")

In [14]:
def export_bstats(dataset_mean, dataset_median, dataset_std, top5, input_dict, variable_abreviation):

    # Generate a timestamp for unique filenames
    start_date = input_dict["start_datetime"][:10]
    end_date = input_dict["end_datetime"][:10]

    # define filenames 
    filename_median = f"median_{variable_abreviation}_{start_date}-{end_date}.tif"
    filename_mean = f"mean_{variable_abreviation}_{start_date}-{end_date}.tif"
    filename_top5 = f"top5_{variable_abreviation}_{start_date}-{end_date}.tif"
    filename_std = f"std_{variable_abreviation}_{start_date}-{end_date}.tif"

    # create filepaths
    output_fp_median = os.path.join(DATA_DIRECTORY, filename_median)
    output_fp_mean = os.path.join(DATA_DIRECTORY, filename_mean)
    output_fp_top5 = os.path.join(DATA_DIRECTORY, filename_top5)
    output_fp_std = os.path.join(DATA_DIRECTORY, filename_std)

    # write to GeoTIFF
    dataset_median.rio.to_raster(DATA_DIRECTORY / output_fp_median)
    dataset_mean.rio.to_raster(DATA_DIRECTORY / output_fp_mean)
    top5.rio.to_raster(DATA_DIRECTORY / output_fp_top5)
    dataset_std.rio.to_raster(DATA_DIRECTORY / output_fp_std)

    print(f"Datsets saved in {DATA_DIRECTORY}:")
    print(f"  - {filename_median}")
    print(f"  - {filename_mean}")
    print(f"  - {filename_top5}")
    print(f"  - {filename_std}")

## Import

In [15]:
# fetching dataset details and loading it into the RAM by calling function
dataset = get_cm_dataset(input_dict).compute()

INFO - 2025-03-09T16:17:42Z - Selected dataset version: "202311"
INFO - 2025-03-09T16:17:42Z - Selected dataset part: "default"



Dataset fetched via API and loaded into RAM


## Pre-processing

In [16]:
# pre-processing dataset by calling function
dataset_preprocessed = pre_processing(dataset, variable_abreviation)


Dataset pre-processing finished


In [17]:
# calculating basic statistics by calling function
dataset_mean, dataset_median, dataset_std = bstats(dataset_preprocessed, variable_abreviation)

Basic statistics calculation finished


In [18]:
# print basic stats (min, max, mean and median
print(f"Minimum {variable_abreviation}: {round(dataset_preprocessed[variable_abreviation].min().item(),2)}{unit}\n"
      f"Maximum {variable_abreviation}: {round(dataset_preprocessed[variable_abreviation].max().item(), 2)}{unit}\n"
      f"Mean {variable_abreviation}: {round(dataset_preprocessed[variable_abreviation].mean().item(), 2)}{unit}\n"
      f"Median {variable_abreviation}: {round(dataset_preprocessed[variable_abreviation].median().item(), 2)}{unit}\n"
     )

Minimum sst: 10.69°C
Maximum sst: 23.05°C
Mean sst: 17.78°C
Median sst: 17.59°C



## Processing

### GETIS ORD G* Statistic

In [19]:
# calculating Getis Ord G* statistics by calling function
getis_statistics = getis_ord_g(dataset_median, variable_abreviation)

Calculation of GETIS-ORD G* statistic finished


### 5th Percentile

In [20]:
# calling function to calculate the median over values > 95th percentile 
top5 = percentile_5th(dataset_preprocessed, variable_abreviation)

/Users/moritzmuhlbauer/micromamba/envs/copernicusmarine/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,



Calculation of 5% highest values finished


### GETIS ORD G* Statistic over 5 % highest values per pixel

In [21]:
# calculating GETIS ORD G* statistic over the median of the 5 % highest values over time per pixel
getis_top5 = getis_ord_g(top5, variable_abreviation)

Calculation of GETIS-ORD G* statistic finished


## Visualization

In [22]:
# function call for plotting and saving basic stats plots 
visualization(dataset_mean, dataset_median, dataset_std, top5, input_dict, variable_abreviation, unit, DOI, temporal_resolution, spatial_resolution)

Plots saved in /Users/moritzmuhlbauer/Copernicus/output/basic_plots:
 - mean_sst_2023-12-01-2024-02-28.png
 - median_sst_2023-12-01-2024-02-28.png
 - std_sst_2023-12-01-2024-02-28.png
 - top5_sst_2023-12-01-2024-02-28.png


In [23]:
# calling visualization function for getis statidtics 
getis_top5_viz(getis_top5, input_dict, variable_abreviation)

Plot of GETIS ORD G* Statistics saved to: /Users/moritzmuhlbauer/Copernicus/output/getis_plots/getis_top5_sst_2023-12-01-2024-02-28.png


In [24]:
# calling visualization function for getis statidtics 
getis_viz(getis_statistics, input_dict, variable_abreviation)

Plot of GETIS ORD G* Statistics saved to: /Users/moritzmuhlbauer/Copernicus/output/getis_plots/getis_entire_dataset_sst_2023-12-01-2024-02-28.png


## Export

In [25]:
export_getis(getis_statistics, getis_top5, input_dict, variable_abreviation)

Datsets saved in /Users/moritzmuhlbauer/Copernicus/output/data:
  - getis_top5_sst_2023-12-01-2024-02-28.gpkg
  - getis_entire_dataset_sst_2023-12-01-2024-02-28.gpkg


In [26]:
export_bstats(dataset_mean, dataset_median, dataset_std, top5, input_dict, variable_abreviation)

Datsets saved in /Users/moritzmuhlbauer/Copernicus/output/data:
  - median_sst_2023-12-01-2024-02-28.tif
  - mean_sst_2023-12-01-2024-02-28.tif
  - top5_sst_2023-12-01-2024-02-28.tif
  - std_sst_2023-12-01-2024-02-28.tif
